# Function Calling
One of the many ways to provide an LLM with real-time, dynamic data is to let it call functions. Or, to be more specifc, let it tell you to call a function for it and then provide the results so it can use it to fulfill a query.

In this notebook, I will show you how to implement function calling using the OpenAI API and GPT-4o-mini to access data from a database to answer a user's request.